# Trabajo Practico Nº1 - Parte 3: Comunicación y Sincronismo

## Información General

* **Universidad Nacional de la Matanza**
* Materia: Programación Concurrente
* Cuatrimestre: Segundo Cuatrimestre, Año 2024
* Profesores:
    * ADAGIO, MATIAS EZEQUIEL
    * CARNUCCIO, ESTEBAN
    * HIRSCHFELDT, DARIO
    * PALOMO, MAXIMO FACUNDO
    * VOLKER MARIANO LEONARDO
* Grupo: M4


* Integrantes:
    * ANTONIOLI, IVÁN OSCAR
    * DI NICCO, LUIS DEMETRIO
    * SANDOVAL VASQUEZ, JUAN LEANDRO
    * TIGANI MARTIN SEBASTIAN
    * VILLCA, LUIS ALBERTO

## Consigna

**Fecha de entrega**: 25/09/2024

**Forma de entrega**:
Se debe generar un informe que contenga los siguientes puntos:

*   **Carátula**: Con los integrantes del grupo.
*   **Link a un repositorio de GitHub**: En donde se encuentre el archivo del cuaderno de Colab generado para este Trabajo Práctico. Este archivo debe ser con la extension ipynb. También se debe subir a GitHub el código fuente.
*   **Conclusiones**: En esta sección se debe describir las dificultadas que encontraron al realizar el trabajo práctico.

Entregar el informe por plataforma de MIeL. Este debe ser en formato .pdf, con nombre TP1_Parte3_NumeroDelGrupo.pdf.

**Enunciado**: Una empresa desea optimizar el espacio utilizado en el edificio donde opera, para esto planean tener un solo baño con capacidad para **tres (3) empleados**, pero dado que actualmente su plantilla de empleados se compone de hombres y mujeres, la empresa quiere garantizar que en el baño no se encuentren hombres y mujeres al mismo tiempo.

Desarrolle un programa que implemente el funcionamiento de dicho baño utilizando algún método de comunicación y/o sincronización.

## Criterios a tener en cuenta

* [Coding Standard](https://google.github.io/styleguide/)
* Ajustes específicos (Ej. llaves en la misma columna)
* Espacios en lugar de tabulaciones (2 espacios)
* Métodos/funciones con menos de 15 líneas.
* Patrones de diseño
* No debe haber números mágicos, uso de constantes descriptivas.

## Resolución

### Código

In [ ]:
%%writefile accesoBaño.cpp
#include <iostream>
#include <mutex>
#include <semaphore>
#include <sstream>
#include <string>
#include <thread>
#include <unistd.h>
#include <vector>

using namespace std;
using namespace chrono;
using namespace this_thread;

const int TIEMPO_ESPERA = 1;
const int TIEMPO_BAÑO = 2;
const int CAPACIDAD_BAÑO = 3;

mutex imprimir_pantalla;
mutex acceso_region_critica;
mutex acceso_region_critica_1;
mutex esperando;
counting_semaphore<CAPACIDAD_BAÑO> turno(CAPACIDAD_BAÑO);
counting_semaphore<CAPACIDAD_BAÑO> salida(CAPACIDAD_BAÑO);

enum Baño
{
  VACIO = 0,
  HAY_HOMBRES = 1,
  HAY_MUJERES = 2
};

int cantidad_mujeres = 0;
int cantidad_hombres = 0;
int situacion_baño = VACIO;
bool espera_mujeres = false;
bool espera_hombres = false;

void mujeres(int numero)
{
  string nombre = "Mujer " + to_string(numero);

  while (true)
  {
    turno.acquire();
    if (situacion_baño == HAY_HOMBRES && acceso_region_critica_1.try_lock())
    {
      espera_mujeres = true;
      while (situacion_baño == HAY_HOMBRES)
      {
        imprimir_pantalla.lock();
        cout << nombre + " esperando" << endl;
        cout << "-----------------------------------------" << endl;
        imprimir_pantalla.unlock();
        sleep_for(seconds(TIEMPO_ESPERA));
      }
      acceso_region_critica_1.unlock();
    }
    if (situacion_baño != HAY_HOMBRES && !espera_hombres)
    {
      acceso_region_critica.lock();
      cantidad_mujeres++;
      if (cantidad_mujeres >= 1)
      {
        situacion_baño = HAY_MUJERES;
        espera_mujeres = false;
      }
      imprimir_pantalla.lock();
      salida.acquire();
      cout << nombre + " entro al baño." << endl;
      cout << "\t- CantidadHombres: " + to_string(cantidad_hombres) + "\n\t- CantidadMujeres: " + to_string(cantidad_mujeres) << endl;
      cout << "-----------------------------------------" << endl;
      imprimir_pantalla.unlock();
      acceso_region_critica.unlock();

      sleep_for(seconds(TIEMPO_BAÑO));

      imprimir_pantalla.lock();
      cout << nombre + " sale del baño." << endl;
      cout << "-----------------------------------------" << endl;
      salida.release();
      imprimir_pantalla.unlock();

      acceso_region_critica.lock();
      cantidad_mujeres--;
      if (cantidad_mujeres == 0)
      {
        situacion_baño = VACIO;
      }
      acceso_region_critica.unlock();
      turno.release();

      break;
    }
    turno.release();
    sleep_for(seconds(TIEMPO_ESPERA));
  }
}

void hombres(int numero)
{
  string nombre = "Hombre " + to_string(numero);

  while (true)
  {
    turno.acquire();
    if (situacion_baño == HAY_MUJERES && acceso_region_critica_1.try_lock())
    {
      espera_hombres = true;
      while (situacion_baño == HAY_MUJERES)
      {
        imprimir_pantalla.lock();
        cout << nombre + " esperando" << endl;
        cout << "-----------------------------------------" << endl;
        imprimir_pantalla.unlock();
        sleep_for(seconds(TIEMPO_ESPERA));
      }
      acceso_region_critica_1.unlock();
    }
    if (situacion_baño != HAY_MUJERES && !espera_mujeres)
    {
      acceso_region_critica.lock();
      cantidad_hombres++;
      if (cantidad_hombres >= 1)
      {
        situacion_baño = HAY_HOMBRES;
        espera_hombres = false;
      }
      imprimir_pantalla.lock();
      salida.acquire();
      cout << nombre + " entro al baño." << endl;
      cout << "\t- CantidadHombres: " + to_string(cantidad_hombres) + "\n\t- CantidadMujeres: " + to_string(cantidad_mujeres) << endl;
      cout << "-----------------------------------------" << endl;
      imprimir_pantalla.unlock();
      acceso_region_critica.unlock();

      sleep_for(seconds(TIEMPO_BAÑO));

      imprimir_pantalla.lock();
      cout << nombre + " sale del baño." << endl;
      cout << "-----------------------------------------" << endl;
      salida.release();
      imprimir_pantalla.unlock();

      acceso_region_critica.lock();
      cantidad_hombres--;
      if (cantidad_hombres == 0)
      {
        situacion_baño = VACIO;
      }
      acceso_region_critica.unlock();
      turno.release();

      break;
    }
    turno.release();
    sleep_for(seconds(TIEMPO_ESPERA));
  }
}

void agregar_hilo(vector<thread> &grupo, int cantidad_personas, void (*funcion_genero)(int))
{
  for (int i = 0; i < cantidad_personas; i++)
  {
    grupo.push_back(thread(funcion_genero, i));
  }
}

void unir_hilos(vector<thread> &grupo)
{
  for (auto &hilo : grupo)
  {
    if (hilo.joinable())
    {
      hilo.join();
    }
  }
}

int main(int argc, char *argv[])
{
  if (argc < 2)
  {
    cout << "Uso: " << argv[0] << " <cantidad de hombres y mujeres>" << endl;
    return EXIT_FAILURE;
  }

  int cantidad_personas = stoi(argv[1]);

  if (cantidad_personas < 0)
  {
    cout << "Ingrese cantidad de hombres y mujeres mayor a 0 " << endl;
    return EXIT_FAILURE;
  }
  cout << "-----------------------------------------" << endl;
  cout << "Comienzo de jornada laboral." << endl;
  cout << "-----------------------------------------" << endl;

  vector<thread> grupo_hombres;
  vector<thread> grupo_mujeres;

  agregar_hilo(grupo_mujeres, cantidad_personas, mujeres);
  agregar_hilo(grupo_hombres, cantidad_personas, hombres);

  unir_hilos(grupo_hombres);
  unir_hilos(grupo_mujeres);

  cout << "Fin de la jornada laboral." << endl;
  cout << "-----------------------------------------\n"
       << endl;

  return EXIT_SUCCESS;
}

Overwriting accesoBaño.cpp


### Compilación

Se le debe indicar al compilador que utilice el estándar C++20, que es una versión específica del lenguaje. En particular, el tipo `counting_semaphore` es una adición reciente introducida, por lo que se debe compilar con el estándar C++20 o posterior.

In [ ]:
!g++ -std=c++20 -o accesoBaño ./accesoBaño.cpp

### Ejecución


In [ ]:
!./accesoBaño 5

-----------------------------------------
Comienzo de jornada laboral.
-----------------------------------------
Mujer 2 entro al baño.
	- CantidadHombres: 0
	- CantidadMujeres: 1
-----------------------------------------
Mujer 1 entro al baño.
	- CantidadHombres: 0
	- CantidadMujeres: 2
-----------------------------------------
Mujer 0 entro al baño.
	- CantidadHombres: 0
	- CantidadMujeres: 3
-----------------------------------------
Mujer 2 sale del baño.
-----------------------------------------
Hombre 2 esperando
-----------------------------------------
Mujer 1 sale del baño.
-----------------------------------------
Mujer 0 sale del baño.
-----------------------------------------
Hombre 4 entro al baño.
	- CantidadHombres: 1
	- CantidadMujeres: 0
-----------------------------------------
Hombre 3 entro al baño.
	- CantidadHombres: 2
	- CantidadMujeres: 0
-----------------------------------------
Hombre 2 entro al baño.
	- CantidadHombres: 3
	- CantidadMujeres: 0
----------------